In [ ]:
!pip install caveclient

In [ ]:
import pandas as pd
import numpy as np
import tqdm

import caveclient

import os

REPO_DIR = f"{os.path.abspath('')}/../"

## Instantiate CAVE client

In [ ]:
my_token = None

In [ ]:
client = caveclient.CAVEclient("minnie65_phase3_v1", auth_token=my_token)

In [ ]:
mat_version = 1293

## Load annotations

In [ ]:
proof_df = client.materialize.query_table("proofreading_status_and_strategy", materialization_version=mat_version)
proof_df = proof_df[proof_df["strategy_axon"] != 'none'].reset_index(drop=True)
proof_df = proof_df[["pt_root_id", "strategy_axon"]].rename(columns={"strategy_axon": "proofreading_strategy"})
proof_df.drop_duplicates("pt_root_id", keep="first", inplace=True)

proof_df["proofreading_strategy"].value_counts()

In [ ]:
region_df = client.materialize.query_table("nucleus_functional_area_assignment", materialization_version=mat_version)
region_df = region_df[["pt_root_id", "tag"]].rename(columns={"tag": "region"})
region_df.drop_duplicates("pt_root_id", keep="first", inplace=True)

region_df["region"].value_counts()

In [ ]:
auto_cell_type_df = client.materialize.query_table("aibs_metamodel_celltypes_v661", materialization_version=mat_version, split_positions=True, desired_resolution=[1, 1, 1])
auto_cell_type_df["classification_system"].replace({"excitatory_neuron": "E", "inhibitory_neuron": "I", "nonneuron": "NONNEURON"}, inplace=True)
auto_cell_type_df = auto_cell_type_df[["pt_position_x", "pt_position_y", "pt_position_z", "pt_root_id", "volume", "classification_system", "cell_type"]]
auto_cell_type_df = auto_cell_type_df.rename(columns={"cell_type": "cell_type_auto", "classification_system": "coarse_cell_type_auto"})
auto_cell_type_df.drop_duplicates("pt_root_id", keep="first", inplace=True)

auto_cell_type_df["coarse_cell_type_auto"].value_counts(), auto_cell_type_df["cell_type_auto"].value_counts()

In [ ]:
manual_cell_type_df = client.materialize.query_table("allen_v1_column_types_slanted_ref", materialization_version=mat_version, split_positions=True, desired_resolution=[1, 1, 1])
manual_cell_type_df = manual_cell_type_df[["pt_root_id", "classification_system", "cell_type"]]
manual_cell_type_df["classification_system"].replace({"aibs_coarse_excitatory": "E", "aibs_coarse_inhibitory": "I", "aibs_coarse_unclear": "UNCLEAR"}, inplace=True)
manual_cell_type_df.rename(columns={"cell_type": "cell_type_manual", "classification_system": "coarse_cell_type_manual"}, inplace=True)
manual_cell_type_df.drop_duplicates("pt_root_id", keep="first", inplace=True)

manual_cell_type_df["coarse_cell_type_manual"].value_counts(), manual_cell_type_df["cell_type_manual"].value_counts()

In [ ]:
cell_df = pd.merge(auto_cell_type_df, manual_cell_type_df, on="pt_root_id", how="left").fillna("")
cell_df = pd.merge(cell_df, region_df, on="pt_root_id", how="left").fillna("")
cell_df = pd.merge(cell_df, proof_df, on="pt_root_id", how="left").fillna("")

In [ ]:
cell_df.to_feather(f"{REPO_DIR}/data/cell_info_{mat_version}.feather", compression="zstd", compression_level=9)